In [ ]:
pwd()

# Start

In [ ]:
# run this ceil if source codes was edited
include("DGP.ji")
# include("qgplsim.ji")
include("MonteCarloSIM.ji")
using Statistics, LinearAlgebra, Clustering
using LaTeXStrings, Plots

import Base.Threads.@threads

# QGPLSIM fitting

In [ ]:
# run this ceil for different data sets

α = [2.0, 1.0] # single index for x
θ = [1.0, -2.0]  # linear for x, <α, θ> = 0 for identification
γ = [1.00, 0.5] # single index for z
β = [0.5, -1.0] # linear for z
# Q(y|x, z) = Xθ + Zβ + g(Xα + Zγ)

n = 600
fx = 2 # y = f(x), fx for different f(x)     
                # fx = 1, y = X*θ + Z*β + (X*α + Z*γ .- 1).^2 + 0.2ϵ
                # fx = 2, y = X*θ + Z*β + log.(1 .+ exp.(X*α + Z*γ .- 1)) + 0.2ϵ

ifz = true # if false, Z = 0, i.e. no category varible
dist = 1 # 0 for gauss, 1 for Cauchy
X, Z, y = dgp(n, γ, θ, α, β, fx, dist,  ifz)
# p = length(α)
# q = length(γ)
τ = 0.5

# Estimation 

model1 = qgplsim.model(X, Z, y, τ)
alpha_type = 1
# different weight of sum on alpha 
# 0 for true alpha, using α in DGP
# 1 for global: ∑ij sum(Bi - Bj) / n;
# 2 for local: ∑z ∑ij (bzi - bzj) * nz / n;
# when Z = 0, global == local
@time alpha, theta, gamma, beta = qgplsim.estimator(model1, alpha_type)
coefa = 1 #/ alpha[1]
coefg = 1 #/ gamma[1]
println("Estimation of index coefs: ")
println("̂α => ", coefa * alpha)
println("̂γ => ", gamma)

println("Estimation of linear coefs: ")
println("̂θ => ", theta)
println("̂β => ", beta)

# theta * alpha

# Paths of $g(x \alpha + z \gamma)$ and $\nabla g$

$\alpha = \hat{\alpha}$,
$z \in S_z$

In [ ]:
using Plots
# include("qgplsim.ji")

In [ ]:
alpha

In [ ]:
alphap = 1.0 * alpha
v = X * alphap # g(Xα + Zγ) => univar func. g(v + zγ) after α was determined and  Z = z
categ, index = model1.categ, model1.index
g, dg = qgplsim.Gz_DGz(v, Z, y, -0.2) # g: g(v), dg: g'(v)
fig = plot()
for m in 1:6
    vm = v[index[categ[m]]]
    gm = g[index[categ[m]]]
    indm = sortperm(vm)
    sorted_gm = gm[indm]
    sorted_vm = vm[indm]

    plot!(fig, sorted_vm, sorted_gm, label = L"g_{%$m} (v)")

end
fig

In [ ]:
fig = plot()
for m in 1:6
    vm = v[index[categ[m]]]
    dgm = dg[index[categ[m]]]
    indm = sortperm(vm)
    sorted_dgm = dgm[indm]
    sorted_vm = vm[indm]
    plot!(fig, sorted_vm, sorted_dgm, label = L"\nabla g_{%$m} (v)")

end
fig

# Monte Carlo Simulation

In [ ]:
Threads.nthreads()

In [ ]:
include("MonteCarloSIM.ji")

In [ ]:
α = [2.0, 1.0] # single index for x
θ = [1.0, -2.0]  # linear for x, <α, θ> = 0 for identification
γ = [1.0, 4.0] # single index for z
β = [1.0, -0.5] # linear for z
# Q(y|x, z) = Xθ + Zβ + g(Xα + Zγ)
n = 300
fx = 1
dist = 1 # Cauchy
len = 200 # MC rept. times
alpha_type = 1

In [ ]:
res, Alpha, Gamma, Theta, Beta = MonteCarlo(n, γ, θ, α, β, fx, dist, len, alpha_type)


In [ ]:
minimum(Alpha)

In [ ]:
histogram(Alpha[:,1], bins = -3:0.1:3)

In [ ]:
histogram(Alpha[:,2], bins = -3:0.1:3)

In [ ]:
histogram(Beta[:, 2], bins = -2:0.1:2)

In [ ]:
Rb = kmeans(transpose(Theta), 1)
Rb.centers

In [ ]:
histogram(Gamma[:,1], bins = -2:0.1:2)

In [ ]:
histogram(Gamma[:,2], bins = -2:0.1:2)

In [ ]:
Rg = kmeans(transpose(Gamma), 2)
Rg.centers

In [ ]:
mean(abs.(Gamma), dims = 1)

In [ ]:
mean(abs.(Alpha), dims = 1)

In [ ]:
ind1 = Rg.assignments .== 1
ind2 = Rg.assignments .== 2
v1 = var(Gamma[ind1, :], dims = 1)

In [ ]:
len1 = sum(ind1) - 1
len2 = sum(ind2) - 1

In [ ]:
mean(Gamma[ind2, :], dims = 1)

In [ ]:
v2 = var(Gamma[ind2, :], dims = 1)

In [ ]:
(v1 * len1 + v2* len2) /(len - 1)

In [ ]:
var(abs.(Gamma), dims = 1)

In [ ]:
Ra= kmeans(transpose(Alpha), 2)
Ra.centers

In [ ]:
using DataFrames
df = DataFrame(α1 = Float64[], α2 = Float64[],
               var_α1 = Float64[], var_α2 = Float64[],    
               θ1 = Float64[], θ2 = Float64[],
               var_θ1 = Float64[], var_θ2 = Float64[], 
               β1 = Float64[], β2 = Float64[],
               var_β1 = Float64[], var_β2 = Float64[], 
               γ1 = Float64[], γ2 = Float64[],
               var_γ1 = Float64[], var_γ2 = Float64[])


In [ ]:
A = [:α1, :α2, :var_α1, :var_α2]
T = [:θ1, :θ2, :var_θ1, :var_θ2]
G = [:γ1, :γ2, :var_γ1, :var_γ2]
B = [:β1, :β2, :var_β1, :var_β2]

In [ ]:
@threads for γ in [[1.0, 0.5], [1.0, 0.001], [0.001, 1.0], [0.5, 1.0],[0.5, 0.5], [0.0001, 0.0001]]
    res, =  MonteCarlo(n, γ, θ, α, β, fx, dist, len, alpha_type)
    push!(df, res)
end